In [1]:
let size = 65536  
var test_large_simd = SIMD[DType.float32, 65536]()
for i in range(0, size):
    test_large_simd[i] = i*2
print(test_large_simd[0])
print(test_large_simd[size-1])


: 

: 

In [ ]:
from sys.info import simdbitwidth
print(simdbitwidth())

256
